In [ ]:
library(ggplot2)
library(dplyr)
library(hrbrthemes)
library(zoo)
library(viridis)
library(tidyr)

options(repr.plot.width = 18, repr.plot.height = 9)


In [ ]:

# Read data
left <- 
right <- 

# Drop 'Unnamed: 0' column
left <- left[, !(names(left) %in% c("Unnamed: 0"))]
right <- right[, !(names(right) %in% c("Unnamed: 0"))]

# Merge data frames on 'IDs'
df <- merge(right, left, by='ids', suffixes=c('_right', '_left'))

# Select columns of interest
cols <- c('Frontal', 'Temporal', 'Hippocampus', 'Frontal_Parietal',
          'Parietal', 'Insula', 'Cingulate', 'Occipital', 'BasalGanglia',
          'Thalamus', 'Cerebellum', 'Ventricles', 'VentralDC', 'WM')

# Create an empty data frame for results
result_df <- data.frame(IDs = df$ids)

# Calculate AI for each specified column
for (col in cols) {
  R <- df[[paste(col, "_right", sep = "")]]
  L <- df[[paste(col, "_left", sep = "")]]

  result_df[[paste(col, "_ai", sep = "")]] <- ((R - L) / (R + L)* 100) 
}



# Reshape data for plotting
df_melted <- result_df %>%
  select(-IDs) %>%
  gather(key = "ROI", value = "AI")

# Plot boxplot
g_free <- ggplot(df_melted, aes(x = ROI, y = AI)) +
  geom_boxplot(fill = "#0099f8") +
  ggtitle("AI across Subjects") +
  theme_minimal() +
  geom_hline(yintercept = 0, color = "black", size = 2, alpha = 0.3)+

  labs(title = 'AI = ((R - L) / (R + L)) * 100',
       x = 'ROI', y = 'AI') +
  theme_minimal(base_size = 20) +
  theme(axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.5),
        #panel.grid.major = element_blank(),
        #panel.grid.minor = element_blank(),
        panel.background = element_blank(),
        axis.line = element_line(color = 'black'))+ylim(-50, 50)

print(g_free)



In [ ]:
result_df %>% 
write.csv("AI_free.csv", row.names = FALSE)

In [ ]:
cat("Al = ((R - L) / (R + L)) * 100\n")

# Calculate mean, median, and IQR for each ROI
mean_df <- apply(result_df[, -1, drop = FALSE], 2, mean)
median_df <- apply(result_df[, -1, drop = FALSE], 2, median)
q1_df <- apply(result_df[, -1, drop = FALSE], 2, function(x) quantile(x, 0.25))
q3_df <- apply(result_df[, -1, drop = FALSE], 2, function(x) quantile(x, 0.75))
iqr_df <- q3_df - q1_df

summary_df <- data.frame(
  ROI = names(median_df),
  Mean = round(mean_df, 2),
  Median = round(median_df, 2),
  Q1 = round(q1_df, 2),
  Q3 = round(q3_df, 2),
  IQR = round(iqr_df, 2)
)

# Display the DataFrame
summary_df